In [ ]:
! pip install shapely

In [3]:
import os
import time
import pandas as pd
import numpy as np
from multiprocessing import Pool
import matplotlib.pyplot as plt
from scipy import stats
import re
import json
import shapely
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

import boto3
from s3fs.core import S3FileSystem


In [5]:
# state names and abbreviations:

names = ['Alabama',
'Arizona',
'Arkansas',
'California',
'Colorado',
'Connecticut',
'Delaware',
'DistrictofColumbia',
'Florida',
'Georgia',
'Hawaii',
'Idaho',
'Illinois',
'Indiana',
'Iowa',
'Kansas',
'Kentucky',
'Louisiana',
'Maine',
'Maryland',
'Massachusetts',
'Michigan',
'Minnesota',
'Mississippi',
'Missouri',
'Montana',
'Nebraska',
'Nevada',
'NewHampshire',
'NewJersey',
'NewMexico',
'NewYork',
'NorthCarolina',
'NorthDakota',
'Ohio',
'Oklahoma',
'Oregon',
'Pennsylvania',
'RhodeIsland',
'SouthCarolina',
'SouthDakota',
'Tennessee',
'Texas',
'Utah',
'Vermont',
'Virginia',
'Washington',
'WestVirginia',
'Wisconsin',
'Wyoming']


abbrv = ['AL',
'AZ',
'AR',
'CA',
'CO',
'CT',
'DE',
'DC',
'FL',
'GA',
'HI',
'ID',
'IL',
'IN',
'IA',
'KS',
'KY',
'LA',
'ME',
'MD',
'MA',
'MI',
'MN',
'MS',
'MO',
'MT',
'NE',
'NV',
'NH',
'NJ',
'NM',
'NY',
'NC',
'ND',
'OH',
'OK',
'OR',
'PA',
'RI',
'SC',
'SD',
'TN',
'TX',
'UT',
'VT',
'VA',
'WA',
'WV',
'WI',
'WY']

# Use sample spectrum output file for testing

In [6]:
spec_df = pd.read_csv('specMapOutput_20180101.csv')
locations = np.asarray((spec_df.latitude, spec_df.longitude)).T
locations.shape
locations[0]

array([ 41.9381141, -88.0906228])

# Take location input and find 1) the state and 2) the county:

Get state centers for location lookup:

In [ ]:
s3 = boto3.resource('s3')
client = boto3.client('s3')
bucket = s3.Bucket('usa-edges-for-speed-mapping')

In [7]:
# Get data from s3 locally.

state_centers = np.zeros((len(names),2))
for i in range(0, len(names)):
    # path for boundary json file
    path = '/home/ec2-user/SageMaker/boundary_data/states/'+names[i]+'.json'
    json_data=open(path, 'r')
    boundary = json.load(json_data)
    state_centers[i, 0] = float(boundary['lat'])
    state_centers[i, 1] = float(boundary['lon'])

County boundary files:

In [8]:
county_files = os.listdir('/home/ec2-user/SageMaker/boundary_data/counties/')

In [9]:
# GET STATE GROUPS~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 

# check time of routine:
t0 = time.time()

path_states = '/home/ec2-user/SageMaker/boundary_data/states/'
path_counties = '/home/ec2-user/SageMaker/boundary_data/counties/'

error_array1 = []
error_array2 = []
#error_array3 = []
#error_array4 = []

state_group1 = []
state_group2 = []
state_group3 = []
#group4 = []

for i in range(0, len(names)):
    try:
        # path for boundary json file
        json_data=open(path_states+names[i]+'.json', 'r')
        boundary = json.load(json_data)
        polygon = boundary['geojson']['coordinates']
        polygon_ar = np.asarray(polygon)[0]
        #plt.fill(polygon_ar[:,0],polygon_ar[:,1],'k', edgecolor='w')
        polygon_ar.shape[1] == 2
        state_group1.append(i)
    except:
        error_array1.append(i)
for i in error_array1:
    try:
        # path for boundary json file
        json_data=open(path_states+names[i]+'.json', 'r')
        boundary = json.load(json_data)
        polygon = boundary['geojson']['coordinates']
        n_poly = len(polygon)
        for n in range(n_poly):
            polygon_ar = np.asarray(polygon[n])[0]
            #plt.fill(polygon_ar[:,0],polygon_ar[:,1],'k', edgecolor='w')
            polygon_ar.shape[1] == 2
        state_group2.append(i)

    except:
        error_array2.append(i)
for i in error_array2:
    try:
        # path for boundary json file
        json_data=open(path_states+names[i]+'.json', 'r')
        boundary = json.load(json_data)
        polygon = boundary['geojson']['coordinates']
        polygon_ar = np.asarray(polygon[0])
        polygon_ar.shape[1] == 2
        #plt.fill(polygon_ar[:,0],polygon_ar[:,1],'k', edgecolor='w')
        state_group3.append(i)

    except:
        #error_array3.append(i)
        print('error for', names[i])
        

        
# FIND STATE/COUNTY PER INPUT LOCATION~~~~~~~~~~~~~~~~~~~~~~~~~~~~
t1 = time.time() - t0

for i in range(0, len(locations)):
    
    t2 = time.time()
    # find nearest states:
    center_diffs = abs(state_centers[:,0] - locations[i][0]) + abs(state_centers[:,1] - locations[i][1])  
    min_index = np.argmin(center_diffs)
    indices = np.argsort(center_diffs)[0:6]
    
    # Check if point is in the 4 nearst states with the nearest state centers:
    for n in indices:
        json_data=open(path_states+names[n]+'.json', 'r')
        boundary = json.load(json_data)
        polygon = boundary['geojson']['coordinates']
        if n in state_group1:
            polygon_ar = np.asarray(polygon)[0]
        elif n in state_group2:
            if n == 16: # kentucky (n=16) has two bounded regions
                polygon_ar = np.asarray(polygon[1])[0]
            else:
                polygon_ar = np.asarray(polygon[0])[0]
        elif n in state_group3:
            polygon_ar = np.asarray(polygon[0])


        polygon_boundary = np.column_stack((polygon_ar[:,0],polygon_ar[:,1])) # Reshape coordinates
        polygon = Polygon(polygon_boundary) # create polygon
        point = Point(locations[i][1], locations[i][0]) # create point
        # check if a point is in the polygon 
        if point.within(polygon) == True:
            break

    state_polygon = polygon
    print('State confirmed as', names[n])
    
    counties = [s for s in county_files if abbrv[n] in s]

    # GIVEN THE STATE, LET'S FIND THE COUNTY:
    county_centers = np.zeros((len(counties),2))
    for j in range(0, len(counties)):
        # path for boundary json file
        json_data=open(path_counties+counties[j], 'r')
        boundary = json.load(json_data)
        county_centers[j, 0] = float(boundary['lat'])
        county_centers[j, 1] = float(boundary['lon'])
                
    error_array1 = []
    error_array2 = []
    error_array3 = []
    error_array4 = []

    group1 = []
    group2 = []
    group3 = []
    group4 = []

    for j in range(0, len(counties)):
        try:
            # path for boundary json file
            json_data=open(path_counties+counties[j], 'r')
            boundary = json.load(json_data)
            nb = len(boundary)
            if nb==2:
                for m in range(nb):
                    polygon = boundary[m]['geojson']['coordinates']
                    polygon_ar = np.asarray(polygon)[0]
                    polygon_ar.shape[1] == 2
                    #plt.fill(polygon_ar[:,0],polygon_ar[:,1],'k', edgecolor='w')
            else:
                polygon = boundary['geojson']['coordinates']
                polygon_ar = np.asarray(polygon)[0]
                polygon_ar.shape[1] == 2
                #plt.fill(polygon_ar[:,0],polygon_ar[:,1],'k', edgecolor='w')
            group1.append(j)
        except:
            error_array1.append(j)
    for j in error_array1:
        try:
            # path for boundary json file
            json_data=open(path_counties+counties[j], 'r')
            boundary = json.load(json_data)
            num = len(boundary['geojson']['coordinates'])
            for n in range(num):
                polygon = boundary['geojson']['coordinates'][n]
                polygon_ar = np.asarray(polygon)[0]
                polygon_ar.shape[1] == 2
                #plt.fill(polygon_ar[:,0],polygon_ar[:,1],'k', edgecolor='w')
            group2.append(j)
        except:
            error_array2.append(j)
    for j in error_array2:
        try:
            # path for boundary json file
            json_data=open(path_counties+counties[j], 'r')
            boundary = json.load(json_data)
            num = len(boundary['geojson']['coordinates'])
            for n in range(num):
                polygon = boundary['geojson']['coordinates'][n]
                polygon_ar = np.asarray(polygon[0])
                polygon_ar.shape[1] == 2
                #plt.fill(polygon_ar[:,0],polygon_ar[:,1],'k', edgecolor='w')
            group3.append(j)
        except:
            error_array3.append(j)
    for j in error_array3:
        try:
            # path for boundary json file
            json_data=open(path_counties+counties[j], 'r')
            boundary = json.load(json_data)
            nb = len(boundary)
            for m in range(nb):
                polygon = boundary['geojson']['coordinates']
                polygon_ar = np.asarray(polygon[0])
                polygon_ar.shape[1] == 2
                #plt.fill(polygon_ar[:,0],polygon_ar[:,1],'k', edgecolor='w')
            group4.append(j)
        except:
            error_array4.append(j)
            
            
    # FIND COUNTY:
    # find nearest counties in state
    center_diffs = abs(county_centers[:,0] - locations[i][0]) + abs(county_centers[:,1] - locations[i][1])  
    min_index = np.argmin(center_diffs)
    indices = np.argsort(center_diffs)

    # Check if point is in the 10 nearest counties with the nearest county centers:
    for n in indices:
        json_data=open(path_counties+counties[n], 'r')
        boundary = json.load(json_data)
        if n in group1:
            polygon = boundary['geojson']['coordinates']
            polygon_ar = np.asarray(polygon)[0]
        elif n in group2:
            polygon = boundary['geojson']['coordinates'][0]
            polygon_ar = np.asarray(polygon)[0]
        elif n in group3:
            polygon = boundary['geojson']['coordinates'][0]
            polygon_ar = np.asarray(polygon[0])
        elif n in group4:
            polygon = boundary['geojson']['coordinates']
            polygon_ar = np.asarray(polygon[0])

        polygon_boundary = np.column_stack((polygon_ar[:,0],polygon_ar[:,1])) # Reshape coordinates
        polygon = Polygon(polygon_boundary) # create polygon
        point = Point(locations[i][1], locations[i][0]) # create point
        # check if a point is in the polygon 
        #print(n, counties[n])
        if point.within(polygon) == True:
            break


    print(i, n, counties[n])
    print("--- %s seconds ---" % (time.time() - t2 + t1))



State confirmed as Illinois
0 89 IL_DuPageCounty.json
--- 0.5018954277038574 seconds ---
State confirmed as Illinois
1 81 IL_CookCounty.json
--- 0.528691291809082 seconds ---
State confirmed as Illinois
2 81 IL_CookCounty.json
--- 0.5002686977386475 seconds ---
State confirmed as Illinois
3 89 IL_DuPageCounty.json
--- 0.4990839958190918 seconds ---
State confirmed as Illinois
4 89 IL_DuPageCounty.json
--- 0.49915289878845215 seconds ---
State confirmed as Illinois
5 81 IL_CookCounty.json
--- 0.5012111663818359 seconds ---
State confirmed as Illinois
6 81 IL_CookCounty.json
--- 0.500786304473877 seconds ---
State confirmed as Illinois
7 89 IL_DuPageCounty.json
--- 0.49946093559265137 seconds ---
State confirmed as Illinois
8 89 IL_DuPageCounty.json
--- 0.4990866184234619 seconds ---
State confirmed as Illinois
9 81 IL_CookCounty.json
--- 0.500683069229126 seconds ---
State confirmed as Illinois
10 89 IL_DuPageCounty.json
--- 0.4990959167480469 seconds ---
State confirmed as Illinois
11 

90 92 GA_BleckleyCounty.json
--- 0.9002828598022461 seconds ---
State confirmed as Georgia
91 92 GA_BleckleyCounty.json
--- 0.9007906913757324 seconds ---
State confirmed as Georgia
92 92 GA_BleckleyCounty.json
--- 0.8995199203491211 seconds ---
State confirmed as Georgia
93 92 GA_BleckleyCounty.json
--- 0.9032082557678223 seconds ---
State confirmed as Georgia
94 92 GA_BleckleyCounty.json
--- 0.9007461071014404 seconds ---
State confirmed as Georgia
95 92 GA_BleckleyCounty.json
--- 0.8740057945251465 seconds ---
State confirmed as Georgia
96 92 GA_BleckleyCounty.json
--- 0.9025847911834717 seconds ---
State confirmed as Nevada
97 3 NV_ClarkCounty.json
--- 0.4944941997528076 seconds ---
State confirmed as Nevada
98 3 NV_ClarkCounty.json
--- 0.522270917892456 seconds ---
State confirmed as Nevada
99 3 NV_ClarkCounty.json
--- 0.4939234256744385 seconds ---
State confirmed as Nevada
100 3 NV_ClarkCounty.json
--- 0.4935774803161621 seconds ---
State confirmed as Nevada
101 3 NV_ClarkCounty

181 20 NC_LenoirCounty.json
--- 0.613793134689331 seconds ---
State confirmed as NorthCarolina
182 20 NC_LenoirCounty.json
--- 0.6132693290710449 seconds ---
State confirmed as NorthCarolina
183 28 NC_BeaufortCounty.json
--- 0.6167974472045898 seconds ---
State confirmed as NorthCarolina
184 28 NC_BeaufortCounty.json
--- 0.615602970123291 seconds ---
State confirmed as NorthCarolina
185 28 NC_BeaufortCounty.json
--- 0.58831787109375 seconds ---
State confirmed as NorthCarolina
186 28 NC_BeaufortCounty.json
--- 0.6162500381469727 seconds ---
State confirmed as NorthCarolina
187 28 NC_BeaufortCounty.json
--- 0.6227920055389404 seconds ---
State confirmed as NorthCarolina
188 28 NC_BeaufortCounty.json
--- 0.620112419128418 seconds ---
State confirmed as NorthCarolina
189 28 NC_BeaufortCounty.json
--- 0.6162629127502441 seconds ---
State confirmed as NorthCarolina
190 28 NC_BeaufortCounty.json
--- 0.6169695854187012 seconds ---
State confirmed as NorthCarolina
191 28 NC_BeaufortCounty.json

268 33 CA_SanMateoCounty.json
--- 0.9642171859741211 seconds ---
State confirmed as California
269 33 CA_SanMateoCounty.json
--- 0.9355571269989014 seconds ---
State confirmed as California
270 33 CA_SanMateoCounty.json
--- 0.9364933967590332 seconds ---
State confirmed as California
271 33 CA_SanMateoCounty.json
--- 0.9362919330596924 seconds ---
State confirmed as California
272 33 CA_SanMateoCounty.json
--- 0.9374046325683594 seconds ---
State confirmed as California
273 33 CA_SanMateoCounty.json
--- 0.965949296951294 seconds ---
State confirmed as California
274 25 CA_SantaCruzCounty.json
--- 0.9376091957092285 seconds ---
State confirmed as California
275 33 CA_SanMateoCounty.json
--- 0.9473013877868652 seconds ---
State confirmed as California
276 33 CA_SanMateoCounty.json
--- 0.9358415603637695 seconds ---
State confirmed as California
277 25 CA_SantaCruzCounty.json
--- 0.9652910232543945 seconds ---
State confirmed as California
278 25 CA_SantaCruzCounty.json
--- 0.935886144638

356 55 GA_ColumbiaCounty.json
--- 0.8959388732910156 seconds ---
State confirmed as Georgia
357 32 GA_RichmondCounty.json
--- 0.8667411804199219 seconds ---
State confirmed as Georgia
358 32 GA_RichmondCounty.json
--- 0.8926246166229248 seconds ---
State confirmed as Georgia
359 55 GA_ColumbiaCounty.json
--- 0.867790699005127 seconds ---
State confirmed as Pennsylvania
360 46 PA_ErieCounty.json
--- 0.6417338848114014 seconds ---
State confirmed as Pennsylvania
361 46 PA_ErieCounty.json
--- 0.6416049003601074 seconds ---
State confirmed as Pennsylvania
362 46 PA_ErieCounty.json
--- 0.6400489807128906 seconds ---
State confirmed as Pennsylvania
363 46 PA_ErieCounty.json
--- 0.6413552761077881 seconds ---
State confirmed as Pennsylvania
364 46 PA_ErieCounty.json
--- 0.6400880813598633 seconds ---
State confirmed as Pennsylvania
365 46 PA_ErieCounty.json
--- 0.6411406993865967 seconds ---
State confirmed as Pennsylvania
366 46 PA_ErieCounty.json
--- 0.6408789157867432 seconds ---
State con

445 44 FL_PalmBeachCounty.json
--- 0.7758572101593018 seconds ---
State confirmed as Florida
446 44 FL_PalmBeachCounty.json
--- 0.7778615951538086 seconds ---
State confirmed as Florida
447 55 FL_MartinCounty.json
--- 0.7656502723693848 seconds ---
State confirmed as Florida
448 55 FL_MartinCounty.json
--- 0.7937917709350586 seconds ---
State confirmed as Florida
449 24 FL_OkeechobeeCounty.json
--- 0.777564525604248 seconds ---
State confirmed as Florida
450 65 FL_DeSotoCounty.json
--- 0.7701504230499268 seconds ---
State confirmed as Florida
451 44 FL_PalmBeachCounty.json
--- 0.7668657302856445 seconds ---
State confirmed as Florida
452 44 FL_PalmBeachCounty.json
--- 0.7648310661315918 seconds ---
State confirmed as Florida
453 44 FL_PalmBeachCounty.json
--- 0.7931768894195557 seconds ---
State confirmed as Florida
454 44 FL_PalmBeachCounty.json
--- 0.765855073928833 seconds ---
State confirmed as Florida
455 44 FL_PalmBeachCounty.json
--- 0.7702052593231201 seconds ---
State confirme

In [ ]:
state_polygon

In [ ]:
polygon

In [ ]:
locations[498]

In [ ]:
boundary

In [ ]:
####################################################################################################################

## Some plotting routines

In [ ]:
for i in range(0, len(county_centers)):
    plt.plot(county_centers[i,1], county_centers[i,0], 'ro')

plt.axis('equal')
plt.show()

In [ ]:
for i in range(0, 50):
    plt.plot(state_centers[i,1], state_centers[i,0], 'ro')

plt.axis('equal')
plt.show()

In [ ]:
locations[0]

In [ ]:
for n in range(0, 50):
    
    print(names[n], abbrv[n])
    county_files = os.listdir('/Users/samtaimourzadeh/python/dev/osm/boundary_data/counties')

    counties = [s for s in county_files if abbrv[n] in s]

    plt.figure(figsize=(11,11))
    error_array1 = []
    error_array2 = []
    error_array3 = []
    error_array4 = []

    group1 = []
    group2 = []
    group3 = []
    group4 = []
    path = '/Users/samtaimourzadeh/python/dev/osm/boundary_data/counties/'

    #for i in range(59, 60):#len(counties)):
    for i in range(0, len(counties)):
        try:
            # path for boundary json file
            json_data=open(path+counties[i], 'r')
            boundary = json.load(json_data)
            #nb = len(boundary)
            if nb==2:
                for m in range(nb):
                    polygon = boundary[m]['geojson']['coordinates']
                    polygon_ar = np.asarray(polygon)[0]
                    plt.fill(polygon_ar[:,0],polygon_ar[:,1],'k', edgecolor='w')
            else:
                polygon = boundary['geojson']['coordinates']
                polygon_ar = np.asarray(polygon)[0]
                plt.fill(polygon_ar[:,0],polygon_ar[:,1],'k', edgecolor='w')
            #polygon = boundary['geojson']['coordinates']
            #polygon_ar = np.asarray(polygon)[0]
            #plt.fill(polygon_ar[:,0],polygon_ar[:,1],'k', edgecolor='w')
            group1.append(i)
        except:
            error_array1.append(i)
    for i in error_array1:
        try:
            # path for boundary json file
            json_data=open(path+counties[i], 'r')
            boundary = json.load(json_data)
            num = len(boundary['geojson']['coordinates'])
            for n in range(num):
                polygon = boundary['geojson']['coordinates'][n]
                polygon_ar = np.asarray(polygon)[0]
                plt.fill(polygon_ar[:,0],polygon_ar[:,1],'k', edgecolor='w')
            group2.append(i)
        except:
            error_array2.append(i)
    for i in error_array2:
        try:
            # path for boundary json file
            json_data=open(path+counties[i], 'r')
            boundary = json.load(json_data)
            num = len(boundary['geojson']['coordinates'])
            for n in range(num):
                polygon = boundary['geojson']['coordinates'][n]
                polygon_ar = np.asarray(polygon[0])
                plt.fill(polygon_ar[:,0],polygon_ar[:,1],'k', edgecolor='w')
            group3.append(i)
        except:
            error_array3.append(i)
    for i in error_array3:
        try:
            # path for boundary json file
            json_data=open(path+counties[i], 'r')
            boundary = json.load(json_data)
            nb = len(boundary)
            for m in range(nb):
                polygon = boundary['geojson']['coordinates']
                polygon_ar = np.asarray(polygon[0])
                plt.fill(polygon_ar[:,0],polygon_ar[:,1],'k', edgecolor='w')
            group4.append(i)
        except:
            error_array4.append(i)
    plt.axis('equal')



    #plt.xlim(-130, -65)
    #plt.ylim(25, 50)

    #plt.xlim(-95, -80)
    #plt.ylim(29, 36)
    #plt.plot(michigan_ar[:,0], michigan_ar[:,1],'k')

    plt.axis('equal')
    plt.show()

In [ ]:
plt.figure(figsize=(11,11))
error_array1 = []
error_array2 = []
error_array3 = []
error_array4 = []

group1 = []
group2 = []
group3 = []
group4 = []

for i in range(0, len(names)):
    try:
        # path for boundary json file
        path = '/Users/samtaimourzadeh/python/dev/osm/boundary_data/states/'
        json_data=open(path+names[i]+'.json', 'r')
        boundary = json.load(json_data)
        polygon = boundary['geojson']['coordinates']
        polygon_ar = np.asarray(polygon)[0]
        plt.fill(polygon_ar[:,0],polygon_ar[:,1],'k', edgecolor='w')
        group1.append(i)
    except:
        error_array1.append(i)
for i in error_array1:
    try:
        # path for boundary json file
        path = '/Users/samtaimourzadeh/python/dev/osm/boundary_data/states/'
        json_data=open(path+names[i]+'.json', 'r')
        boundary = json.load(json_data)
        polygon = boundary['geojson']['coordinates']
        polygon_ar = np.asarray(polygon[0])[0]
        plt.fill(polygon_ar[:,0],polygon_ar[:,1],'k', edgecolor='w')
        group2.append(i)

    except:
        error_array2.append(i)
for i in error_array2:
    try:
        # path for boundary json file
        path = '/Users/samtaimourzadeh/python/dev/osm/boundary_data/states/'
        json_data=open(path+names[i]+'.json', 'r')
        boundary = json.load(json_data)
        polygon = boundary['geojson']['coordinates']
        polygon_ar = np.asarray(polygon[0])
        plt.fill(polygon_ar[:,0],polygon_ar[:,1],'k', edgecolor='w')
        group3.append(i)

    except:
        error_array3.append(i)
for i in error_array3:
    try:
        # path for boundary json file
        path = '/Users/samtaimourzadeh/python/dev/osm/boundary_data/states/'
        json_data=open(path+names[i]+'.json', 'r')
        boundary = json.load(json_data)
        polygon = boundary['geojson']['coordinates']
        polygon_ar = np.asarray(polygon[0])
        plt.fill(polygon_ar[:,0],polygon_ar[:,1],'k', edgecolor='w')
        group4.append(i)

    except:
        error_array4.append(i)

# path for boundary json file
# kentucky has two boundries
path = '/Users/samtaimourzadeh/python/dev/osm/boundary_data/states/'
json_data=open(path+names[16]+'.json', 'r')
boundary = json.load(json_data)
polygon = boundary['geojson']['coordinates']
polygon_ar = np.asarray(polygon[1])[0]
plt.fill(polygon_ar[:,0],polygon_ar[:,1],'k', edgecolor='w')

#Michigan
#path = '/Users/samtaimourzadeh/python/dev/osm/boundary_data/states/'
#json_data=open(path+names[21]+'.json', 'r')
#boundary = json.load(json_data)
#polygon = boundary['geojson']['coordinates']
#polygon_ar = np.asarray(polygon[0])
#plt.fill(polygon_ar[:,0],polygon_ar[:,1],'k', edgecolor='w')

plt.axis('equal')
plt.xlim(-130, -65)
plt.ylim(25, 50)
plt.show()

In [ ]:
names[10]

In [ ]:
group2

In [ ]:
spec_df